In [1]:
import numpy as np
import pandas as pd

import warnings as wn
wn.filterwarnings('ignore')

In [2]:
# this function gives intrabucket correlation value, for now this function is not dynamic
def intrabucket_correl(column, index,curve = 'Same'):
    
    if  column == index and curve == 'Same':
        correlation = 1
        return correlation
    
    elif column == index and curve == 'Different':
        return 0.999
    
    elif column != index and curve == 'Same':
        num = abs(column- index)
        denom = min(column, index)
        exponent_term = np.exp(-0.03*(num/denom))
        rho = max(0.40,exponent_term)
        return rho
    
    else:
        num = abs(column- index)
        denom = min(column, index)
        exponent_term = np.exp(-0.03*(num/denom))
        rho = max(0.40,exponent_term)
        return rho*0.999
                
correl = intrabucket_correl(5,3,'Different')
correl

0.9792184746334485

In [3]:
# weighted net sensitivities is filled in a matrix , net sensitivities * risk weight gives this matrix values
WSk_euro = np.array([[0.00], [-2.57], [-3.856], [0.00], [4.84], [0.00]])
Sb_euro = WSk_euro.sum() # this is used in interbucket aggregation
Sb_euro_matrix = np.full((3,3),Sb_euro)  # this is used in intrabucket aggregation

WSk_usd = np.array([[-1.927],
                  [-1.767],
                  [0.00]])

Sb_usd = WSk_usd.sum()  # this is used in interbucket aggregation
Sb_usd_matrix = np.full((3,3),Sb_usd) # this is used in intrabucket aggregation


In [4]:
# EURO BUCKET
correl = intrabucket_correl(3,5)
rho = np.full((WSk_euro.size,WSk_euro.size), correl)
np.fill_diagonal(rho,1)
rho
rho_df = pd.DataFrame(rho,columns = [3,5,10,3,5,10], index= [3,5,10,3,5,10])
rho_df.iloc[[[2,0],[0,2]]] = intrabucket_correl(3,10) 
rho_df.iloc[[[2,1],[1,2]]] = intrabucket_correl(5,10)
rho_df.iloc[3:6,0:3] =  rho_df.iloc[0:3,0:3].applymap(lambda x : x * 0.999)
rho_df.iloc[0:3,3:6] =  rho_df.iloc[0:3,0:3].applymap(lambda x : x * 0.999)
rho_df.iloc[3:6,3:6] =  rho_df.iloc[0:3,0:3].applymap(lambda x : x * 1)
(rho_df.style.set_caption('Intra-bucket correlation scenario for Euro'))


,3,5,10,3,5,10
3,1.000000,0.980199,0.932394,0.999000,0.979218,0.931461
5,0.980199,1.000000,0.970446,0.979218,0.999000,0.969475
10,0.932394,0.970446,1.000000,0.931461,0.969475,0.999000
3,0.999000,0.979218,0.931461,1.000000,0.980199,0.932394
5,0.979218,0.999000,0.969475,0.980199,1.000000,0.970446
10,0.931461,0.969475,0.999000,0.932394,0.970446,1.000000


In [5]:
# USD BUCKET 
correl = intrabucket_correl(3,5)
rho = np.full((WSk_usd.size,WSk_usd.size), correl)
np.fill_diagonal(rho,1)
rho_df_usd = pd.DataFrame(rho,columns = [3,5,10], index= [3,5,10])
rho_df_usd.iloc[[[2,0],[0,2]]] = intrabucket_correl(3,10) 
rho_df_usd.iloc[[[2,1],[1,2]]] = intrabucket_correl(5,10)

(rho_df_usd.style.set_caption('Intra-bucket correlation scenario for USD'))


,3,5,10
3,1.000000,0.980199,0.932394
5,0.980199,1.000000,0.970446
10,0.932394,0.970446,1.000000


In [6]:
## Correlation Scenarios for EURO
Base_scenario_eur = rho_df.applymap(lambda x : x*1)
High_scenario_eur = rho_df.applymap(lambda x : min(x*1.25,1))
Low_scenario_eur = rho_df.applymap(lambda x : max(x*2 - 1 , 0.75 * x))
Low_scenario_eur.style.set_caption('Intra-bucket correlation for Low correlation scenario ')

,3,5,10,3,5,10
3,1.000000,0.960397,0.864788,0.998000,0.958437,0.862923
5,0.960397,1.000000,0.940891,0.958437,0.998000,0.938950
10,0.864788,0.940891,1.000000,0.862923,0.938950,0.998000
3,0.998000,0.958437,0.862923,1.000000,0.960397,0.864788
5,0.958437,0.998000,0.938950,0.960397,1.000000,0.940891
10,0.862923,0.938950,0.998000,0.864788,0.940891,1.000000


In [7]:
## Correlation Scenarios for USD
Base_scenario_usd = rho_df_usd.applymap(lambda x : x*1)
High_scenario_usd = rho_df_usd.applymap(lambda x : min(x*1.25,1))
Low_scenario_usd = rho_df_usd.applymap(lambda x : max(x*2 - 1 , 0.75 * x))
Base_scenario_usd.style.set_caption('Intra-bucket correlation for Baseline scenario ')

,3,5,10
3,1.000000,0.980199,0.932394
5,0.980199,1.000000,0.970446
10,0.932394,0.970446,1.000000


In [8]:
"""
We have the weighted net sensitivity matrix(WSk) and intrabucket correlation matrix above 
now we move to bucket level capital charge for Euro bucket for three correlation(rho) scenarios and store it in a array

"""
Kb_base = np.sqrt(WSk_euro.T@Base_scenario_eur@WSk_euro)
Kb_low = np.sqrt(WSk_euro.T@Low_scenario_eur@WSk_euro)
Kb_high = np.sqrt(WSk_euro.T@High_scenario_eur@WSk_euro)
Kb_matrix_eur = np.array([Kb_base, Kb_low, Kb_high])
Kb_matrix_eur


array([[[1.75894403]],

       [[1.91634345]],

       [[1.586     ]]])

In [9]:
"""
We have the weighted net sensitivity matrix(WSk) and intrabucket correlation matrix above 
now we move to bucket level capital charge for Euro bucket for three correlation(rho) scenarios and store it in a array

"""
Kb_base = np.sqrt(WSk_usd.T@Base_scenario_usd@WSk_usd)
Kb_low = np.sqrt(WSk_usd.T@Low_scenario_usd@WSk_usd)
Kb_high = np.sqrt(WSk_usd.T@High_scenario_usd@WSk_usd)
Kb_matrix_usd = np.array([Kb_base, Kb_low, Kb_high])
print(Kb_matrix_usd)

[[[3.67570246]]

 [[3.65731339]]

 [[3.694     ]]]


In [10]:
"""
As we have the bucket level capital charge 
now we move to interbucket capital charge
in our case we have two buckets(Euro,USD)
final capital charge is the maximum of three gamma scenarios

"""

def calculate_capital_charge(Kb_matrix_eur, Kb_matrix_usd, Sb_euro, Sb_usd, gamma):
    gamma_coefficients = {"base": 0.5, "low": 0.375, "high": 0.625}  # interbucket correlation according to FRTB 
    
    gamma_value = gamma_coefficients[gamma]
    capital = Kb_matrix_eur**2 + Kb_matrix_usd**2 + 2 * Sb_euro * Sb_usd * gamma_value
    return capital


gamma_values = ["base", "low", "high"] 


for gamma, Kb_eur, Kb_usd in zip(gamma_values, Kb_matrix_eur, Kb_matrix_usd):
    capital = calculate_capital_charge(Kb_eur, Kb_usd, Sb_euro, Sb_usd, gamma)
    
    print(f"Gamma: {gamma}, Capital Charge: {capital}")

Gamma: base, Capital Charge: [[22.46335672]]
Gamma: low, Capital Charge: [[21.44232644]]
Gamma: high, Capital Charge: [[23.484387]]
